<h1><strong>Voting and Averaging Classifier</strong> </h1> 
    
<h3>Acá se va a estar probando tanto el ensamblaje de Voting como de Averaging con los distintos algoritmos implementados.</h3>

Primero se hacen los imports necesarios

In [4]:
import pandas as pd
import numpy as np

import sklearn

from sklearn.naive_bayes  import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import svm
from catboost import CatBoostClassifier
                        
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score, accuracy_score


from feature_builder import process_dataset, add_text_embeddings, calculate_keyword_encoding
from hyperparameter_tuning import random_search



In [5]:
train_dataset = pd.read_csv('train.csv')

In [6]:
test_dataset = pd.read_csv('test.csv')

In [7]:
y = train_dataset.loc[:,'target']

<h2>Preparo los distintos sets con features diferentes.</h2>
Los distintos algoritmos necesitan diferentes sets, según lo investigado.

Primero los procesados completos con spacy

In [8]:
x_processed = process_dataset(train_dataset, use_spacy=True)

Percentage of words covered in the embeddings = 0.4937444933920705
Percentage of words covered in the embeddings = 0.5961027457927369


In [9]:
x_train_processed, x_test_processed, y_train_processed, y_test_processed = train_test_split(x_processed, y, test_size = .30, random_state = 17)

Ahora los que solo necesitan embeddings. 

In [10]:
x_embedd = train_dataset.copy()
add_text_embeddings(x_embedd, x_embedd['text'], 'embeddings')
x_embedd.drop(['text', 'location', 'keyword', 'id', 'target'], axis=1, inplace=True)

Percentage of words covered in the embeddings = 0.4937444933920705


In [11]:
x_train_embedd, x_test_embedd, y_train_embedd, y_test_embedd = train_test_split(x_embedd, y, test_size = .30, random_state = 17)

<h2> Defino los algoritmos a usar
    

In [12]:
logisticRegr = LogisticRegression(solver='liblinear', penalty='l1', multi_class='auto', max_iter=1000, C=1)

In [31]:
SVC = svm.SVC(degree=10,coef0=10,C=5, probability=True)

In [14]:
catboost = CatBoostClassifier(verbose=False)

In [15]:
MultiNB = MultinomialNB()

<h1> Empiezo las pruebas de Voting

------------------------------------------------------------------------------------------------------
<h3>PODRIAMOS PROBAR UNO QUE HAGA TODAS LAS COMBINACIONES Y DEVUELVA EL MEJOR, QUIEN SE ANIMA</h3>
----------------------------------------------------------------------------------------------------------------

<h3> Logistic, SVC, CatBoost

In [52]:
VC_LSC = VotingClassifier(estimators=[('lr', logisticRegr), ('svc', SVC), ('catboost', catboost)], voting='hard')

Pruebo procesadas

In [53]:
VC_LSC.fit(x_train_processed, y_train_processed)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('svc',
                              SVC(C=5, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=10,
                                  decision_function_shape='ovr', degree=10,
                                  gamm

In [54]:
y_pred_processed_LSC = VC_LSC.predict(x_test_processed)

In [55]:
f1_score(y_test_processed, y_pred_processed_LSC)

0.784796573875803

0.7869198312236287 - los 3 - proccessed sin spacy

<h3> Logistic, CatBoost

In [56]:
VC_LSC = VotingClassifier(estimators=[('lr', logisticRegr), ('catboost', catboost)], voting='hard')

Pruebo procesadas

In [57]:
VC_LSC.fit(x_train_processed, y_train_processed)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f3591d35850>)],
                 flatten_transform=True, n_jobs=None, voting='hard',
                 weights=None)

In [58]:
y_pred_processed_LSC = VC_LSC.predict(x_test_processed)

In [59]:
f1_score(y_test_processed, y_pred_processed_LSC)

0.7770915216170691

<h1> Ahora las pruebas de Averaging

<h3> LogisticRegr, SVM, Catboost

In [43]:
AC_LSC = VotingClassifier(estimators=[('lr', logisticRegr), ('svc', SVC), ('catboost', catboost)], voting='soft')

In [44]:
AC_LSC.fit(x_train_processed, y_train_processed)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('svc',
                              SVC(C=5, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=10,
                                  decision_function_shape='ovr', degree=10,
                                  gamm

In [45]:
y_pred_processed_LSC = AC_LSC.predict(x_test_processed)

In [46]:
f1_score(y_test_processed, y_pred_processed_LSC)

0.7830744509908945

<h3>LR y CB

In [47]:
AC_LC = VotingClassifier(estimators=[('lr', logisticRegr), ('catboost', catboost)], voting='soft')

In [48]:
AC_LC.fit(x_train_processed, y_train_processed)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f3591d35850>)],
                 flatten_transform=True, n_jobs=None, voting='soft',
                 weights=None)

In [49]:
y_pred_processed_LC = AC_LC.predict(x_test_processed)

In [50]:
f1_score(y_test_processed, y_pred_processed_LC)

0.7925729442970822

<h2>Dado el resultado, paso a exportar el cvs con el AC entrenado con todo el train_set.

In [22]:
x_test_proccesed = process_dataset(test_dataset, use_spacy=True)

Percentage of words covered in the embeddings = 0.5707598689343111
Percentage of words covered in the embeddings = 0.665389037945573


In [23]:
AC_LSC.fit(x_processed, y)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l1',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7f3591d35850>)],
                 flatten_transform=True, n_jobs=None, voting='soft',
                 weights=None)

In [24]:
y_pred = AC_LSC.predict(x_test_proccesed)

In [25]:
ids = test_dataset['id']
final_df = pd.DataFrame({'target': [x for x in y_pred]}, index=ids)

In [26]:
final_df.to_csv('AC-CB-LR-processed.csv')